## Imports y configuración

In [ ]:
import asyncio
import pandas as pd
from src.core.domains.products.catalog_service import CatalogRetriever, CATEGORY_DESCRIPTIONS
from src.chains.assistant.schemas import ConversationSlots, UserIntent

# Inicializar CatalogRetriever
catalog_retriever = CatalogRetriever("repuesto_motos_mundibot")
print("✅ CatalogRetriever inicializado correctamente")

## Validar conexión y obtener categorías reales


In [ ]:
# Probar conexión básica
try:
    brands = await catalog_retriever.get_unique_brands()
    print(f"✅ Conexión exitosa - {len(brands)} marcas encontradas")
    
    # Obtener categorías reales de la colección
    categories_with_desc = await catalog_retriever.get_unique_categories()
    real_categories = [cat['name'] for cat in categories_with_desc]
    
    print(f"✅ {len(real_categories)} categorías encontradas en la colección")
    print("\n📂 Categorías reales en la colección:")
    for i, cat in enumerate(sorted(real_categories), 1):
        print(f"  {i:2d}. {cat}")
        
except Exception as e:
    print(f"❌ Error de conexión: {e}")

## Comparar categorías reales vs descripciones semánticas

In [ ]:
# Categorías en CATEGORY_DESCRIPTIONS
semantic_categories = set(CATEGORY_DESCRIPTIONS.keys())
real_categories_set = set(real_categories)

print("🔍 ANÁLISIS DE COBERTURA DE CATEGORÍAS")
print("=" * 60)

# Categorías que están en la colección pero NO en las descripciones
missing_descriptions = real_categories_set - semantic_categories
if missing_descriptions:
    print(f"\n❌ FALTAN DESCRIPCIONES ({len(missing_descriptions)}):")
    for cat in sorted(missing_descriptions):
        print(f"  • {cat}")
else:
    print("\n✅ Todas las categorías reales tienen descripción semántica")

# Categorías que están en las descripciones pero NO en la colección
extra_descriptions = semantic_categories - real_categories_set
if extra_descriptions:
    print(f"\n⚠️  DESCRIPCIONES EXTRA ({len(extra_descriptions)}):")
    for cat in sorted(extra_descriptions):
        print(f"  • {cat}")
else:
    print("\n✅ No hay descripciones semánticas extra")

# Categorías que coinciden perfectamente
matching_categories = real_categories_set & semantic_categories
print(f"\n✅ CATEGORÍAS COINCIDENTES ({len(matching_categories)}):")
for cat in sorted(matching_categories):
    print(f"  • {cat}")

# Resumen
print(f"\n📊 RESUMEN:")
print(f"  • Categorías en colección: {len(real_categories_set)}")
print(f"  • Descripciones semánticas: {len(semantic_categories)}")
print(f"  • Coincidencias: {len(matching_categories)}")
print(f"  • Cobertura: {len(matching_categories)/len(real_categories_set)*100:.1f}%")

## Validar funcionalidad completa del CatalogRetriever

In [ ]:
print("🧪 PRUEBAS DE FUNCIONALIDAD DEL CATALOGRETRIEVER")
print("=" * 60)

# Test 1: Marcas únicas
try:
    brands = await catalog_retriever.get_unique_brands()
    print(f"✅ get_unique_brands(): {len(brands)} marcas")
    print(f"   Primeras 5: {brands[:5]}")
except Exception as e:
    print(f"❌ get_unique_brands() falló: {e}")

# Test 2: Modelos por marca (YAMAHA)
try:
    yamaha_models = await catalog_retriever.get_models_by_brand("YAMAHA")
    print(f"✅ get_models_by_brand('YAMAHA'): {len(yamaha_models)} modelos")
    print(f"   Primeros 5: {yamaha_models[:5]}")
except Exception as e:
    print(f"❌ get_models_by_brand('YAMAHA') falló: {e}")

# Test 3: Categorías con descripciones
try:
    categories = await catalog_retriever.get_unique_categories()
    print(f"✅ get_unique_categories(): {len(categories)} categorías con descripciones")
    print(f"   Primera categoría: {categories[0]['name']}")
    print(f"   Descripción: {categories[0]['description'][:80]}...")
except Exception as e:
    print(f"❌ get_unique_categories() falló: {e}")

# Test 4: Subcategorías por categoría
try:
    frenos_subcats = await catalog_retriever.get_subcategories_by_category("FRENOS")
    print(f"✅ get_subcategories_by_category('FRENOS'): {len(frenos_subcats)} subcategorías")
    print(f"   Primeras 3: {frenos_subcats[:3]}")
except Exception as e:
    print(f"❌ get_subcategories_by_category('FRENOS') falló: {e}")

# Test 5: Obtener todos los catálogos en paralelo
try:
    import time
    start_time = time.time()
    all_catalogs = await catalog_retriever.get_all_catalogs()
    end_time = time.time()
    
    print(f"✅ get_all_catalogs(): {(end_time-start_time)*1000:.1f}ms")
    print(f"   Total marcas: {all_catalogs['total_brands']}")
    print(f"   Total categorías: {all_catalogs['total_categories']}")
except Exception as e:
    print(f"❌ get_all_catalogs() falló: {e}")

## Validar descripciones semánticas específicas

In [ ]:
print("📖 VALIDACIÓN DE DESCRIPCIONES SEMÁNTICAS")
print("=" * 60)

# Categorías más importantes para validar
categorias_importantes = [
    "ELECTRICO / ENCENDIDO",
    "FRENOS", 
    "MOTOR INTERNO",
    "TRANSMISION SECUNDARIA",
    "LLANTA",
    "CARROCERIA / PLASTICOS"
]

for categoria in categorias_importantes:
    if categoria in CATEGORY_DESCRIPTIONS:
        desc = CATEGORY_DESCRIPTIONS[categoria]
        print(f"\n🔧 **{categoria}**")
        print(f"   Longitud: {len(desc)} caracteres")
        print(f"   Descripción: {desc}")
        
        # Verificar si la categoría existe en la colección real
        if categoria in real_categories_set:
            print(f"   ✅ Existe en la colección")
        else:
            print(f"   ❌ NO existe en la colección")
    else:
        print(f"\n❌ **{categoria}** - SIN DESCRIPCIÓN SEMÁNTICA")

print(f"\n📊 Total descripciones validadas: {len(categorias_importantes)}")

## Test de performance y cache

In [ ]:
print("⚡ PRUEBAS DE PERFORMANCE Y CACHE")
print("=" * 60)

import time

# Test sin cache
catalog_retriever.clear_cache()
print("🗑️ Cache limpiado")

# Primera ejecución (sin cache)
start_time = time.time()
brands_1 = await catalog_retriever.get_unique_brands(use_cache=True)
categories_1 = await catalog_retriever.get_unique_categories(use_cache=True)
time_1 = (time.time() - start_time) * 1000

print(f"⏱️  Primera ejecución (sin cache): {time_1:.1f}ms")
print(f"   Marcas: {len(brands_1)}, Categorías: {len(categories_1)}")

# Segunda ejecución (con cache)
start_time = time.time()
brands_2 = await catalog_retriever.get_unique_brands(use_cache=True)
categories_2 = await catalog_retriever.get_unique_categories(use_cache=True)
time_2 = (time.time() - start_time) * 1000

print(f"⏱️  Segunda ejecución (con cache): {time_2:.1f}ms")
print(f"   Marcas: {len(brands_2)}, Categorías: {len(categories_2)}")

# Verificar que los resultados son idénticos
brands_match = brands_1 == brands_2
categories_match = len(categories_1) == len(categories_2)

print(f"\n✅ Resultados idénticos: Marcas={brands_match}, Categorías={categories_match}")
print(f"🚀 Mejora de performance: {time_1/time_2:.1f}x más rápido con cache")

In [ ]:
# Celda 1: Validar configuración de la colección
from qdrant_client import QdrantClient
from src.core.domains.products.schemas import PRODUCTS_SCHEMA
import json

# Conectar a Qdrant
client = QdrantClient("localhost", port=6333)
collection_name = "repuesto_motos_mundibot"

print("🔍 VALIDANDO CONFIGURACIÓN DE LA COLECCIÓN")
print("=" * 60)

try:
    # Obtener información de la colección
    collection_info = client.get_collection(collection_name)
    
    print(f"📊 Información de la colección '{collection_name}':")
    print(f"  • Estado: {collection_info.status}")
    print(f"  • Total de puntos: {collection_info.points_count:,}")
    print(f"  • Vectores configurados: {len(collection_info.config.params.vectors)}")
    
    # Mostrar configuración de vectores
    print(f"\n🎯 Configuración de Vectores:")
    for vector_name, vector_config in collection_info.config.params.vectors.items():
        print(f"  • {vector_name}:")
        print(f"    - Tamaño: {vector_config.size}")
        print(f"    - Distancia: {vector_config.distance}")
    
    # Validar payload schema
    payload_schema = collection_info.config.payload_schema
    
    print(f"\n📋 Payload Schema Configurado:")
    if payload_schema:
        print(f"  • Total campos indexados: {len(payload_schema)}")
        
        # Comparar con PRODUCTS_SCHEMA esperado
        expected_fields = set(PRODUCTS_SCHEMA.keys())
        configured_fields = set(payload_schema.keys())
        
        missing_fields = expected_fields - configured_fields
        extra_fields = configured_fields - expected_fields
        matching_fields = expected_fields & configured_fields
        
        print(f"  • Campos coincidentes: {len(matching_fields)}")
        print(f"  • Campos faltantes: {len(missing_fields)}")
        print(f"  • Campos extra: {len(extra_fields)}")
        
        if missing_fields:
            print(f"\n❌ CAMPOS FALTANTES:")
            for field in sorted(missing_fields):
                expected_type = PRODUCTS_SCHEMA[field]
                print(f"    • {field} ({expected_type})")
        
        if extra_fields:
            print(f"\n⚠️  CAMPOS EXTRA (no en PRODUCTS_SCHEMA):")
            for field in sorted(extra_fields):
                configured_type = payload_schema[field]
                print(f"    • {field} ({configured_type})")
        
        print(f"\n✅ CAMPOS CORRECTAMENTE CONFIGURADOS:")
        for field in sorted(matching_fields):
            expected_type = PRODUCTS_SCHEMA[field]
            configured_type = payload_schema[field]
            match_status = "✅" if str(expected_type) == str(configured_type) else "⚠️"
            print(f"    {match_status} {field}: {configured_type} (esperado: {expected_type})")
    
    else:
        print("  ❌ No hay payload schema configurado")
    
    # Validar con muestra de datos
    print(f"\n🔍 VALIDANDO CON MUESTRA DE DATOS:")
    
    # Obtener algunos puntos para ver la estructura real
    sample_points = client.scroll(
        collection_name=collection_name,
        limit=3,
        with_payload=True,
        with_vectors=False
    )
    
    if sample_points[0]:
        sample_point = sample_points[0][0]
        actual_payload = sample_point.payload
        
        print(f"  • Campos en datos reales: {len(actual_payload)}")
        print(f"  • Muestra de campos:")
        
        for i, (field, value) in enumerate(list(actual_payload.items())[:10]):
            value_type = type(value).__name__
            is_indexed = field in (payload_schema or {})
            index_status = "🔍" if is_indexed else "❌"
            print(f"    {index_status} {field}: {value_type} = {str(value)[:50]}...")
            
        # Verificar campos críticos
        critical_fields = ["marca", "categoria", "precio", "es_llanta", "marcas_lista"]
        print(f"\n🎯 CAMPOS CRÍTICOS:")
        for field in critical_fields:
            if field in actual_payload:
                value = actual_payload[field]
                is_indexed = field in (payload_schema or {})
                status = "✅" if is_indexed else "❌"
                print(f"    {status} {field}: {type(value).__name__} = {value}")
            else:
                print(f"    ❌ {field}: FALTANTE en datos")
    
    else:
        print("  ❌ No hay datos en la colección para validar")

except Exception as e:
    print(f"❌ Error accediendo a la colección: {e}")

In [ ]:
# Celda 2: Inspección detallada de la configuración
print("\n🔧 INSPECCIÓN DETALLADA DE CONFIGURACIÓN")
print("=" * 60)

try:
    # Obtener configuración completa
    collection_info = client.get_collection(collection_name)
    
    # Convertir a dict para mejor visualización
    config_dict = {
        "collection_name": collection_name,
        "status": collection_info.status,
        "points_count": collection_info.points_count,
        "config": {
            "vectors": {},
            "payload_schema": {},
            "optimizer_config": collection_info.config.optimizer_config,
            "wal_config": collection_info.config.wal_config,
            "hnsw_config": collection_info.config.hnsw_config
        }
    }
    
    # Extraer configuración de vectores
    for vector_name, vector_config in collection_info.config.params.vectors.items():
        config_dict["config"]["vectors"][vector_name] = {
            "size": vector_config.size,
            "distance": str(vector_config.distance),
            "on_disk": getattr(vector_config, 'on_disk', None)
        }
    
    # Extraer payload schema
    if collection_info.config.payload_schema:
        for field_name, field_type in collection_info.config.payload_schema.items():
            config_dict["config"]["payload_schema"][field_name] = str(field_type)
    
    # Mostrar configuración formateada
    print(json.dumps(config_dict, indent=2, ensure_ascii=False))
    
    # Estadísticas de uso
    print(f"\n📈 ESTADÍSTICAS DE USO:")
    print(f"  • Colección: {collection_name}")
    print(f"  • Puntos totales: {collection_info.points_count:,}")
    print(f"  • Vectores por tipo:")
    
    for vector_name in config_dict["config"]["vectors"]:
        print(f"    - {vector_name}: {collection_info.points_count:,} vectores")
    
    print(f"  • Campos indexados: {len(config_dict['config']['payload_schema'])}")
    print(f"  • Memoria estimada: ~{collection_info.points_count * 1536 * 4 / 1024 / 1024:.1f} MB (solo vectores densos)")

except Exception as e:
    print(f"❌ Error en inspección detallada: {e}")